In [2]:
import os
import CONSTANTS as c
import UTILS as u
import XMLUTILS as xu

In [3]:
def process_annotation(annotation, img_filepath):
    for object_ in annotation.findall('object'):
        name = ((object_.find('name')).text)
        corners = ([int(corner.text) for corner in list(object_.find('bndbox'))])
        data = '{},{},{},{},{},{}'.format(img_filepath, corners[0], corners[1], corners[2], corners[3], name)
        u.append_to_file(c.ANN_PATH, data)

In [4]:
# clear annotation file
u.remove_file_contents(c.ANN_PATH)

#iterate through all files
for root, dirs, files in os.walk(c.BASE_DIR):
    for file in sorted(files):
        filepath = root + '/' + file
        
        # as a bounus, remove all the automatically created .DS_Store's
        if file == '.DS_Store':
            os.unlink(filepath)

        # only allow images in the standardized format that have been annotated
        if file[-4:] == c.XML_EXT and 'xml_preds' not in filepath and any([train_dir in filepath for train_dir in c.TRAIN_VID_KEYS]):
            annotation = xu.open_xml_file(filepath)
            img_filepath = filepath.replace('Volumes', 'home').replace('xmls', 'imgs').replace(c.XML_EXT, c.IMG_EXT)
            process_annotation(annotation, img_filepath)
            print(filepath + ": done")

/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-00-00.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-00-15.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-00-25.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-00-30.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-00-35.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-00-45.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-00-50.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-00-55.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-01-00.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-02-30.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-02-35.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-02-40.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_

/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-41-50.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-42-20.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-43-30.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-44-35.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-44-40.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-45-55.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-46-20.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-47-20.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-47-25.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-48-30.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-49-15.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_04HD/xmls/D0232_04HD_00-51-15.xml: done
/Volumes/nyee/datasets/frcnn_data/D0232_

/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-06-50.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-06-55.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-07-00.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-07-05.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-07-10.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-07-15.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-07-20.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-07-25.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-07-30.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-07-35.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-07-40.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-07-45.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_

/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-15-35.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-15-40.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-15-45.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-15-50.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-15-55.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-16-00.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-16-05.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-16-10.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-16-15.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-16-20.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-16-25.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-16-30.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_

/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-24-45.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-24-50.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-24-55.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-25-00.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-25-05.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-25-10.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-25-15.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-25-20.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-25-25.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-25-30.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-25-35.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_04HD/xmls/D0673_04HD_00-25-40.xml: done
/Volumes/nyee/datasets/frcnn_data/D0673_

...

/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-24-45.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-24-50.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-24-55.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-25-00.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-25-05.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-25-10.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-25-15.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-25-20.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-25-25.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-25-30.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-25-35.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-25-40.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_

/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-33-20.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-33-25.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-33-30.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-33-35.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-33-40.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-33-45.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-33-50.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-33-55.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-34-00.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-34-05.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-34-10.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-34-15.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_

/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-42-10.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-42-15.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-42-20.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-42-25.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-42-30.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-42-35.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-42-40.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-42-45.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-42-50.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-42-55.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-43-00.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_09HD/xmls/D0772_09HD_00-43-05.xml: done
/Volumes/nyee/datasets/frcnn_data/D0772_

/Volumes/nyee/datasets/frcnn_data/D008_03HD/xmls/D008_03HD_00-25-10.xml: done
/Volumes/nyee/datasets/frcnn_data/D008_03HD/xmls/D008_03HD_00-25-15.xml: done
/Volumes/nyee/datasets/frcnn_data/D008_03HD/xmls/D008_03HD_00-26-25.xml: done
/Volumes/nyee/datasets/frcnn_data/D008_03HD/xmls/D008_03HD_00-26-55.xml: done
/Volumes/nyee/datasets/frcnn_data/D008_03HD/xmls/D008_03HD_00-27-25.xml: done
/Volumes/nyee/datasets/frcnn_data/D008_03HD/xmls/D008_03HD_00-27-35.xml: done
/Volumes/nyee/datasets/frcnn_data/D008_03HD/xmls/D008_03HD_00-28-15.xml: done
/Volumes/nyee/datasets/frcnn_data/D008_03HD/xmls/D008_03HD_00-29-25.xml: done
/Volumes/nyee/datasets/frcnn_data/D008_03HD/xmls/D008_03HD_00-29-40.xml: done
/Volumes/nyee/datasets/frcnn_data/D008_03HD/xmls/D008_03HD_00-29-45.xml: done
/Volumes/nyee/datasets/frcnn_data/D008_03HD/xmls/D008_03HD_00-30-00.xml: done
/Volumes/nyee/datasets/frcnn_data/D008_03HD/xmls/D008_03HD_00-30-05.xml: done
/Volumes/nyee/datasets/frcnn_data/D008_03HD/xmls/D008_03HD_00-31

/Volumes/nyee/datasets/frcnn_data/D0442_06HD/xmls/D0442_06HD_00-16-25.xml: done
/Volumes/nyee/datasets/frcnn_data/D0442_06HD/xmls/D0442_06HD_00-17-45.xml: done
/Volumes/nyee/datasets/frcnn_data/D0442_06HD/xmls/D0442_06HD_00-17-55.xml: done
/Volumes/nyee/datasets/frcnn_data/D0442_06HD/xmls/D0442_06HD_00-18-00.xml: done
/Volumes/nyee/datasets/frcnn_data/D0442_06HD/xmls/D0442_06HD_00-18-05.xml: done
/Volumes/nyee/datasets/frcnn_data/D0442_06HD/xmls/D0442_06HD_00-18-15.xml: done
/Volumes/nyee/datasets/frcnn_data/D0442_06HD/xmls/D0442_06HD_00-18-50.xml: done
/Volumes/nyee/datasets/frcnn_data/D0442_06HD/xmls/D0442_06HD_00-19-25.xml: done
/Volumes/nyee/datasets/frcnn_data/D0442_06HD/xmls/D0442_06HD_00-20-05.xml: done
/Volumes/nyee/datasets/frcnn_data/D0442_06HD/xmls/D0442_06HD_00-20-15.xml: done
/Volumes/nyee/datasets/frcnn_data/D0442_06HD/xmls/D0442_06HD_00-22-05.xml: done
/Volumes/nyee/datasets/frcnn_data/D0442_06HD/xmls/D0442_06HD_00-22-25.xml: done
/Volumes/nyee/datasets/frcnn_data/D0442_